In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
!git clone -q https://github.com/coleman-zachery/poker_pytorch.git git_download
!rm -rf libs
!mv git_download/libs .
!rm -rf git_download

from libs.constants import RANKS, SUITS
from libs.helpers import card_strings_to_tensor, card_str_to_tuple, classify_poker_hand, poker_hand_label_to_index, reverse_poker_hand_index
from libs.encoders import CardEncoder, PokerHandClassifier
from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [ ]:
generate_four_of_a_kind()

In [ ]:
generate_straight_flush()

In [14]:
# ---- training set ----
card_strings = [
    ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
    ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
    ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
    ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
    ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
    ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
    ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
    ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
    ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
    ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
    ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
    ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
    ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
    ["__", "__", "__", "__", "__"], # empty hand
]


In [22]:
# create tensors and labels
card_tensors = []
labels = []
for hand in card_strings:
    card_tensor = card_strings_to_tensor(hand)
    card_tensors.append(card_tensor)
    card_tuples = [card_str_to_tuple(cs) for cs in hand]
    label = classify_poker_hand(card_tuples)
    labels.append(poker_hand_label_to_index(label))

# get classification embeddings
card_tensors = torch.stack(card_tensors)
labels = torch.tensor(labels, dtype=torch.long)
# print("Card Tensors:\n", card_tensors)
# print("Labels:\n", labels)
encoder = CardEncoder()
classifier = PokerHandClassifier() # ---- choose classifier ----
encoded_cards = encoder(card_tensors.view(-1, 2))
encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
outputs = classifier(encoded_hands)
print("Classifier Outputs:\n", outputs)

[12, 13, 14, 15]
[11, 12, 13, 14, 15]
[6, 7, 8, 9, 10]
[4, 4, 4, 4, 10]
[5, 5, 5, 10, 10]
[3, 6, 10, 12, 14]
[3, 4, 5, 6, 7]
[3, 6, 8, 8, 8]
[5, 5, 9, 9, 14]
[4, 7, 10, 12, 12]
[3, 5, 6, 8, 11]
[6, 6]
[5, 5, 10, 10]
[]
Classifier Outputs:
 tensor([[-9.0355e-02, -2.8867e-01, -7.3524e-02,  1.2004e-01,  2.5445e-04,
          1.4808e-01,  1.2241e-01,  2.0916e-01, -1.2041e-01, -5.8306e-03,
         -1.7682e-01],
        [-8.2164e-02, -2.4992e-01,  1.3818e-01,  4.3440e-02, -3.5071e-02,
          5.7853e-02,  4.6342e-02,  1.3140e-01, -1.9046e-01, -2.8658e-02,
         -2.3023e-01],
        [-8.8056e-02, -1.3002e-02,  5.9981e-03, -8.1940e-02,  1.4762e-01,
          1.0607e-01, -2.6327e-01,  1.7589e-01, -1.1138e-01,  1.0355e-01,
         -2.9653e-01],
        [-1.1471e-01, -2.0171e-01,  4.7763e-01,  7.7727e-02, -5.1406e-02,
         -2.5071e-01, -9.1137e-02,  2.0489e-01, -2.0169e-01,  1.1071e-01,
         -3.0686e-01],
        [ 3.5800e-03, -2.0836e-01,  2.5172e-03,  1.0437e-02, -8.1188e-02,
  

In [19]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


Epoch [1/100], Loss: 0.0000
Epoch [2/100], Loss: 0.0000
Epoch [3/100], Loss: 0.0000
Epoch [4/100], Loss: 0.0000
Epoch [5/100], Loss: 0.0000
Epoch [6/100], Loss: 0.0000
Epoch [7/100], Loss: 0.0000
Epoch [8/100], Loss: 0.0000
Epoch [9/100], Loss: 0.0000
Epoch [10/100], Loss: 0.0000
Epoch [11/100], Loss: 0.0000
Epoch [12/100], Loss: 0.0000
Epoch [13/100], Loss: 0.0000
Epoch [14/100], Loss: 0.0000
Epoch [15/100], Loss: 0.0000
Epoch [16/100], Loss: 0.0000
Epoch [17/100], Loss: 0.0000
Epoch [18/100], Loss: 0.0000
Epoch [19/100], Loss: 0.0000
Epoch [20/100], Loss: 0.0000
Epoch [21/100], Loss: 0.0000
Epoch [22/100], Loss: 0.0000
Epoch [23/100], Loss: 0.0000
Epoch [24/100], Loss: 0.0000
Epoch [25/100], Loss: 0.0000
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0000
Epoch [28/100], Loss: 0.0000
Epoch [29/100], Loss: 0.0000
Epoch [30/100], Loss: 0.0000
Epoch [31/100], Loss: 0.0000
Epoch [32/100], Loss: 0.0000
Epoch [33/100], Loss: 0.0000
Epoch [34/100], Loss: 0.0000
Epoch [35/100], Loss: 0

In [20]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [21]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")



hand: ['A♥', 'K♦', 'Q♣', 'J♠', '__']
correct label confidence: high card -- 100.00%

hand: ['A♥', 'K♥', 'Q♥', 'J♥', 'X♥']
correct label confidence: straight flush -- 100.00%

hand: ['9♠', '8♠', '7♠', '6♠', '5♠']
correct label confidence: straight flush -- 100.00%

hand: ['3♦', '3♣', '3♥', '3♠', '9♥']
correct label confidence: four of a kind -- 100.00%

hand: ['4♥', '4♠', '4♣', '9♦', '9♥']
correct label confidence: full house -- 100.00%

hand: ['2♣', '5♣', '9♣', 'J♣', 'K♣']
correct label confidence: flush -- 100.00%

hand: ['6♥', '5♠', '4♦', '3♣', '2♥']
correct label confidence: straight -- 100.00%

hand: ['7♠', '7♥', '7♣', '2♦', '5♥']
correct label confidence: three of a kind -- 100.00%

hand: ['8♦', '8♣', '4♥', '4♠', 'K♥']
correct label confidence: two pair -- 100.00%

hand: ['J♣', 'J♠', '3♥', '6♦', '9♣']
correct label confidence: one pair -- 100.00%

hand: ['5♥', 'X♠', '7♣', '4♦', '2♥']
correct label confidence: high card -- 100.00%

hand: ['5♥', '5♠', '__', '__', '__']
correct labe

In [ ]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")